# RNN : 情意分析

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#### ● IMDB 互聯網電影資料庫(Internate Movie Database)：
#### 是一個關於電影、電影演員、電視節目、電視明星、電視製作的數據庫，包含許多訊息，例如：評論、內容、演員、片長…
#### ●此處用的數據集為來自imdb中25000條電影評論，以（情意）正面、反面作為輸出，keras 已經幫我們預處理，下圖為原始資料的模樣（非此情意分析的原始資料）

### 原始data模樣（非此情意分析的原始資料）
![說明](v2-d5421cd5c9a908bb0f5c7d1d5af3161d_r.jpg)

In [2]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.datasets import imdb

## Step 1 讀檔

#### ● num_words=10000 決定幾個常用字

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

#### ● 通常一個字（或詞）我們會給他一個代表數字，通常是以這個字出現的頻率作為他的代表數字

In [22]:
x_train[0]

array([1415,   33,    6,   22,   12,  215,   28,   77,   52,    5,   14,
        407,   16,   82,    2,    8,    4,  107,  117, 5952,   15,  256,
          4,    2,    7, 3766,    5,  723,   36,   71,   43,  530,  476,
         26,  400,  317,   46,    7,    4,    2, 1029,   13,  104,   88,
          4,  381,   15,  297,   98,   32, 2071,   56,   26,  141,    6,
        194, 7486,   18,    4,  226,   22,   21,  134,  476,   26,  480,
          5,  144,   30, 5535,   18,   51,   36,   28,  224,   92,   25,
        104,    4,  226,   65,   16,   38, 1334,   88,   12,   16,  283,
          5,   16, 4472,  113,  103,   32,   15,   16, 5345,   19,  178,
         32])

????? 資料長怎樣

#### ● x_train中的東西長度不同

In [6]:
#len(x_test)
#len(x_train[0])
len(x_train[1])

189

In [7]:
#y_train[0] #great comments
y_train[0] #bad comments

1

## Step 2 資料前處理
#### ● 由於每份資料的長度不同，所以為了電腦計算速度考量我們將資料長度都調成一致
#### ● pad 就是將長度不足的資料補０，長度過長的資料縮短
#### ● 此處由於輸入為one-hot encoding 所以不用正規劃  ??????

In [8]:
x_train = sequence.pad_sequences(x_train, maxlen=100)
x_test = sequence.pad_sequences(x_test, maxlen=100)

## Step 3 打造神經網路

In [9]:
model=Sequential()

In [10]:
# one-hot encoding => embedding  #壓縮維度維128維
model.add(Embedding(10000, 128))

#### ● dropout : 選到的不參加學習，避免overfitting (所有NN都可以用)
#### e.g. 今天有一場考試，老師已事先給了考古題，而每個同學分別分配一題去背答案(training)，所以我們要避免此情況，因此dropout就是限制佔多少%的人不能進來答題(training)，讓大家都認真學習(training)
#### ● recurrent_dropout = 0.2 :  hidden state 也要設dropout， 避免讓hidden state的輸出進來 ，避免overfitting

In [11]:
#不用給activation fn，因為LSTM的activation fn固定為tanh
#不用給控制閘函數，因為LSTM的控制閘函數固定為sigmoid
#不一定要128
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))

In [12]:
#輸出一個數即可，另一個自動生成
#由於此處是情境分析，所以只有正評與負評 e.g.正評0.3與負評0.7
model.add(Dense(1, activation='sigmoid'))

#### ● binary_crossentropy : 常用於分類問題
#### ●此處輸入只有0、1因此為binary_crossentropy

In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#1280000 = 128*10000 (embedding不會做bias)
#131584 = (128 (embedding) + 128(LSTM) + 1(bias) ) * 4 (3個gates加上自己長得像RNN) * 128(LSTM有128神經元) 
#下圖說明

### 解釋參數數量
![說明](ss.jpg)

## Step 4 Training

#### ●我們在乎testing的誤差，不在乎training的誤差
#### ●validation_data=(x_test, y_test)每次學習完，都會用測試資料算誤差，可以知道每次testing的誤差
#### ●val_loss越來越大代表為overfitting

In [16]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 81s 3ms/sample - loss: 0.4769 - accuracy: 0.7708 - val_loss: 0.3901 - val_accuracy: 0.8281
Epoch 2/10
25000/25000 [==============================] - 83s 3ms/sample - loss: 0.3318 - accuracy: 0.8626 - val_loss: 0.3810 - val_accuracy: 0.8359
Epoch 3/10
25000/25000 [==============================] - 83s 3ms/sample - loss: 0.2665 - accuracy: 0.8912 - val_loss: 0.4028 - val_accuracy: 0.8292
Epoch 4/10
25000/25000 [==============================] - 83s 3ms/sample - loss: 0.2168 - accuracy: 0.9156 - val_loss: 0.4187 - val_accuracy: 0.8402
Epoch 5/10
25000/25000 [==============================] - 83s 3ms/sample - loss: 0.1748 - accuracy: 0.9338 - val_loss: 0.4476 - val_accuracy: 0.8350
Epoch 6/10
25000/25000 [==============================] - 83s 3ms/sample - loss: 0.1352 - accuracy: 0.9499 - val_loss: 0.4889 - val_accuracy: 0.8348
Epoch 7/10
25000/25000 [==============================] 

## Step 5 儲存weights : weights是我們要學習的東西

In [18]:
#json : 是一種資料存放格式  (jupyter notebook就是用json存起來的)
model_json = model.to_json()

#'imdb_model_architecture.json'亂取的，用來說明我們此處存的是網路架構
#'w'是python標準open的檔案
open('imdb_model_architecture.json','w').write(model_json)

#儲存權重
model.save_weights('imdb_model_weights.h5')

## ？？？？能做甚麼應用